In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Conv2D, Dense, Flatten, TimeDistributed, Input

# IMU Data (Acceleration, Gyro) - Time Series
imu_input = Input(shape=(50, 6))  # 50 time steps, 6 IMU features
imu_lstm = LSTM(128, return_sequences=True)(imu_input)
imu_lstm = LSTM(64)(imu_lstm)

# Optical Flow Data (CNN Feature Extraction)
image_input = Input(shape=(224, 224, 3))  # Image sequence
cnn = Conv2D(32, (3,3), activation='relu')(image_input)
cnn = Flatten()(cnn)
cnn = Dense(64, activation='relu')(cnn)

# Merge CNN & LSTM Outputs
merged = tf.keras.layers.concatenate([imu_lstm, cnn])
output = Dense(3, activation='linear')(merged)  # Predict X, Y, Z displacement

# Model
model = Model(inputs=[imu_input, image_input], outputs=output)
model.compile(optimizer='adam', loss='mse')
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 50, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 222, 222,  │        896 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 50, 128)   │     69,120 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 1577088)   │          0 │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     49,408 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │ 100,933,6… │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128)       │          0 │ lstm_1[0][0],     │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 3)         │        387 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 101,053,507 (385.49 MB)

 Trainable params: 101,053,507 (385.49 MB)

 Non-trainable params: 0 (0.00 B)